# GPU Accelerated Options Pricing via Monte Carlo Simulation and Deep Learning
---
This will be a project to price European-style options using a few different methods:

1. A baseline using the Black-Scholes formula
2. A monte carlo simulation ran on CPU
3. The simulation accelerated on GPU
4. A neural network surrogate model trained on the simulations, prioritizing both inference speed and accuracy.


# Black-Scholes Baseline

In [17]:
import numpy as np
from scipy.stats import norm
import time
import torch
import torch.nn as nn

In [2]:
s = 100
k = 105
t = 1
r = 0.05
sigma = 0.2

In [3]:
def black_scholes(s, k, r, t, sigma):
  # s is the sport price of an asset
  # k is the strike price
  # r is the risk-free interest rate
  # t is the time to maturity
  # sigma is the volatility of the asset though,
  # this can not be directly observed in the market
  d1 = (np.log(s/k) + (r + (sigma**2/2)) * t) / (sigma * np.sqrt(t))
  d2 = d1 - sigma * np.sqrt(t)

  call_price = norm.cdf(d1) * s - norm.cdf(d2) * k * np.exp((-r)*t)
  return call_price

print(black_scholes(s=s, k=k, t=t, r=r, sigma=sigma))

8.021352235143176


# Monte Carlo Simulation

In [4]:
def simulate_path(s, t, r, sigma):
    # There are ~252 trading days in a year
    steps = 252 * t
    dt = 1.0 / 252

    price_path = [s]
    current_price = s

    for _ in range(steps):
        # Generate a random shock
        z = np.random.randn()

        next_price = current_price * np.exp((r - sigma**2/2) * dt + sigma * np.sqrt(dt) * z)

        price_path.append(next_price)
        current_price = next_price

    return price_path

In [5]:
def monte_carlo_price(s, k, t, r, sigma, simulations=10000):
    payoffs = []
    start_time = time.perf_counter()
    for _ in range(simulations):
        # Simulate one path and get the final price
        path = simulate_path(s, t, r, sigma)
        final_price = path[-1]

        # Calculate the payoff for this path
        payoff = np.maximum(final_price - k, 0)

        payoffs.append(payoff)
    end_time = time.perf_counter()
    exec_time = end_time - start_time
    print(f"Python CPU time: {exec_time:.4f} seconds")
    avg_payoff = np.mean(payoffs)
    option_price = avg_payoff * np.exp((-r)*t)
    return option_price

In [6]:
print("Option price: ",  monte_carlo_price(s=s, k=k, t=t, r=r, sigma=sigma))

Python CPU time: 8.3630 seconds
Option price:  8.15449460808497


# Monte Carlo w/ GPU Acceleration

In [7]:
!pip install pybind11 nvcc4jupyter

In [8]:
%%writefile mc.cu
#include <iostream>
#include <stdio.h>
#include <cmath>
#include <curand_kernel.h>

__global__ void monte_carlo_kernel(float *payoffs, int simulations, float s, float k, float t, float r, float sigma){
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < simulations){
        int steps = max(1, (int)lrintf(252.0 * t));
        float dt = t / steps;
        float current_price = s;

        curandState_t state;
        curand_init(tid, 0, 0, &state);

        for(int i=0; i<steps; i++){
            float z = curand_normal(&state);
            float next_price = current_price * expf((r - (sigma * sigma)/2) * dt + sigma * sqrtf(dt) * z);
            current_price = next_price;
        }

        payoffs[tid] = fmaxf(current_price - k, 0);
    }
}

int main(){
    int simulations = pow(2,20);
    float s = 100.0f, k = 105.0f, t = 1.0f, r = 0.05f, sigma = 0.2f;

    // Allocate GPU memory
    float *g_payoffs;
    cudaMalloc((void**)&g_payoffs, simulations * sizeof(float));

    // Kernel default config
    int threads_per_block = 256;
    int num_blocks = (simulations + threads_per_block - 1) / threads_per_block;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);

    monte_carlo_kernel<<<num_blocks, threads_per_block>>>(g_payoffs, simulations, s, k, t, r, sigma);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms = 0;
    cudaEventElapsedTime(&ms, start, stop);
    std::cout << "GPU Kernel Time: " << ms / 1000 << " seconds" <<std::endl;
    // Copy results over to CPU
    float *c_payoffs = new float[simulations];
    cudaMemcpy(c_payoffs, g_payoffs, simulations * sizeof(float), cudaMemcpyDeviceToHost);

    double sum = 0.0;
    for (int i = 0; i < simulations; i++){
        sum += c_payoffs[i];
    }
    double avg_payoff = sum / simulations;
    double option_price = avg_payoff * expf(-r * t);

    std::cout<<"Option Price (GPU): " << option_price;

    delete[] c_payoffs;
    cudaFree(g_payoffs);

}

Overwriting mc.cu


In [9]:
!nvcc -arch=sm_89 mc.cu -o mc && ./mc

GPU Kernel Time: 0.00143974 seconds
Option Price (GPU): 8.01112

The CPU ran 10,000 simulations in about 8.25 seconds

The GPU ran 1,048,576 simulations in about 1.4 milliseconds

10,000/8.25 = about 1,212 simulations a second
1,048,576/0.0014 = about 748 million

The speedup was about 600,000x!

In [10]:
%%writefile monte_carlo_pybind.cu
#include <pybind11/pybind11.h>
#include <cuda_runtime.h>
#include <curand_kernel.h>
#include <cmath>

namespace py = pybind11;

__global__ void monte_carlo_kernel(float *payoffs, int simulations, float s, float k, float t, float r, float sigma){
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < simulations){
        curandState_t state;
        curand_init(tid, 0, 0, &state);

        int steps = max(1, (int)lrintf(252.0f * t));
        float dt = t / steps;
        float current_price = s;

        for(int i = 0; i < steps; i++){
            float z = curand_normal(&state);
            float next_price = current_price * expf((r - (sigma * sigma)/2.0f) * dt + sigma * sqrtf(dt) * z);
            current_price = next_price;
        }
        payoffs[tid] = fmaxf(current_price - k, 0.0f);
    }
}

double monte_carlo_cuda_py(float s, float k, float t, float r, float sigma, int simulations) {
    float *d_payoffs;
    cudaMalloc((void**)&d_payoffs, simulations * sizeof(float));

    int threadsPerBlock = 256;
    int numBlocks = (simulations + threadsPerBlock - 1) / threadsPerBlock;
    monte_carlo_kernel<<<numBlocks, threadsPerBlock>>>(d_payoffs, simulations, s, k, t, r, sigma);
    cudaDeviceSynchronize();

    float *h_payoffs = new float[simulations];
    cudaMemcpy(h_payoffs, d_payoffs, simulations * sizeof(float), cudaMemcpyDeviceToHost);

    double sum = 0.0;
    for (int i = 0; i < simulations; i++) {
        sum += h_payoffs[i];
    }
    double avg_payoff = sum / simulations;
    double option_price = avg_payoff * exp(-r * t);

    delete[] h_payoffs;
    cudaFree(d_payoffs);

    return option_price;
}

PYBIND11_MODULE(monte_carlo_pybind, m) {
    m.def("price_option", &monte_carlo_cuda_py, "CUDA-accelerated Monte Carlo Option Pricer");
}

Overwriting monte_carlo_pybind.cu


In [11]:
!nvcc -O3 -shared -std=c++20 -Xcompiler -fPIC -arch=sm_89 `python3 -m pybind11 --includes` monte_carlo_pybind.cu -o monte_carlo_pybind`python3-config --extension-suffix`

# Surrogate Neural Network

In [12]:
import importlib.util, sys

spec = importlib.util.spec_from_file_location("monte_carlo_pybind", "./monte_carlo_pybind.cpython-310-x86_64-linux-gnu.so")
mod = importlib.util.module_from_spec(spec); spec.loader.exec_module(mod)

monte_carlo_pybind = mod

In [13]:
import time

start_time = time.perf_counter()
gpu_price = monte_carlo_pybind.price_option(100.0, 105.0, 1.0, 0.05, 0.2, 262_144)
end_time = time.perf_counter()

print(f"Option Price (GPU via Pybind11): {gpu_price}")
print(f"Execution Time: {end_time - start_time:.4f} seconds")

Option Price (GPU via Pybind11): 8.038570376207446
Execution Time: 0.1038 seconds


In [14]:
BUCKET_DEFINITIONS = {
    "micro": {
        "s_range": (0.5, 5.0),
        "moneyness": (0.5, 2.0),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.40, 1.20),
    },
    "low": {
        "s_range": (5.0, 20.0),
        "moneyness": (0.5, 2.0),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.35, 0.90),
    },
    "upper_mid": {
        "s_range": (20.0, 50.0),
        "moneyness": (0.6, 2.0),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.20, 0.60),
    },
    "mid": {
        "s_range": (50.0, 150.0),
        "moneyness": (0.6, 2.0),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.20, 0.60),
    },
    "high": {
        "s_range": (150.0, 500.0),
        "moneyness": (0.6, 1.5),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.15, 0.45),
    },
    "mega": {
        "s_range": (500.0, 2000.0),
        "moneyness": (0.7, 1.3),
        "t_choices": [1/252, 5/252, 10/252, 21/252, 63/252, 126/252, 252/252],
        "r_choices": (0.02, 0.05),
        "sigma_range": (0.15, 0.45),
    },
}


In [15]:
def generate_sample(bucket_name):
  rules = BUCKET_DEFINITIONS[bucket_name]

  s_range = rules['s_range']
  moneyness_range = rules['moneyness']
  t_choices = rules['t_choices']
  r_choices = rules['r_choices']
  sigma_range = rules['sigma_range']

  s = np.random.uniform(*s_range)
  moneyness = np.random.uniform(*moneyness_range)
  k = s * moneyness

  t = np.random.choice(t_choices)
  # Fix: Generate a single float for r within the range
  r = np.random.uniform(*r_choices)


  sigma = np.random.uniform(*sigma_range)

  return s, k, t, r, sigma

In [16]:
num_samples = 1_500_000
x = []
y = []

bucket_names = list(BUCKET_DEFINITIONS.keys())
print("Starting sample generation")
for i in range(num_samples):
  name = np.random.choice(bucket_names)
  s, k, t, r, sigma = generate_sample(name)
  price =  monte_carlo_pybind.price_option(s, k ,t, r, sigma, 262_144)
  x.append([s, k, t, r, sigma])
  y.append(price)

  if(i+1)%100_000 == 0:
    print(f"Generated {i + 1}/{num_samples} samples")

x_set = np.array(x)
y_set = np.array(y)
np.save("x_set.npy", x_set)
np.save("y_set.npy", y_set)

print(f"Successfully generated {len(x_set)} samples.")

Starting sample generation
Generated 100000/1500000 samples
Generated 200000/1500000 samples
Generated 300000/1500000 samples
Generated 400000/1500000 samples
Generated 500000/1500000 samples
Generated 600000/1500000 samples
Generated 700000/1500000 samples
Generated 800000/1500000 samples
Generated 900000/1500000 samples
Generated 1000000/1500000 samples
Generated 1100000/1500000 samples
Generated 1200000/1500000 samples
Generated 1300000/1500000 samples
Generated 1400000/1500000 samples
Generated 1500000/1500000 samples
Successfully generated 1500000 samples.


In [87]:
class OptionPricer(nn.Module):
  def __init__(self):
    super(OptionPricer, self).__init__()
    self.layer1 = nn.Linear(5, 256)
    self.activation1 = nn.ReLU()
    self.layer2 = nn.Linear(256, 256)
    self.activation2 = nn.ReLU()
    self.layer3 = nn.Linear(256, 256)
    self.activation3 = nn.ReLU()
    self.output_layer = nn.Linear(256, 1)

  def forward(self, x):
    x = self.activation1(self.layer1(x))
    x = self.activation2(self.layer2(x))
    x = self.activation3(self.layer3(x))
    x = self.output_layer(x)
    return x

In [88]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler


print("Loading and preparing data...")
X_data = np.load("x_set.npy")
y_data = np.load("y_set.npy").reshape(-1, 1) # Reshape for loss function

X_tensor = torch.tensor(X_data, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.15, random_state=42)

from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train)
test_dataset = TensorDataset(X_test_tensor, y_test)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
print("Data is ready.")

model = OptionPricer()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

Loading and preparing data...
Data is ready.


In [90]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Starting training on {device}...")

epochs = 100
for epoch in range(epochs):
    model.train()
    for train_inputs, train_labels in train_loader:
        train_inputs, train_labels = train_inputs.to(device), train_labels.to(device)

        optimizer.zero_grad()

        predictions = model(train_inputs)

        loss = loss_fn(predictions, train_labels)

        loss.backward()

        optimizer.step()

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            predictions = model(test_inputs)
            test_loss += loss_fn(predictions, test_labels).item()

    avg_test_loss = test_loss / len(test_loader)
    print(f"Epoch {epoch+1}/{epochs}, Test MSE Loss: ${avg_test_loss:.4f}")

print("--- Training Finished ---")

Starting training on cuda...
Epoch 1/100, Test MSE Loss: $44.0938
Epoch 2/100, Test MSE Loss: $14.5988
Epoch 3/100, Test MSE Loss: $7.2303
Epoch 4/100, Test MSE Loss: $4.4811
Epoch 5/100, Test MSE Loss: $3.0567
Epoch 6/100, Test MSE Loss: $2.3567
Epoch 7/100, Test MSE Loss: $1.9487
Epoch 8/100, Test MSE Loss: $1.4263
Epoch 9/100, Test MSE Loss: $1.2054
Epoch 10/100, Test MSE Loss: $0.9312
Epoch 11/100, Test MSE Loss: $1.0388
Epoch 12/100, Test MSE Loss: $0.7090
Epoch 13/100, Test MSE Loss: $0.6126
Epoch 14/100, Test MSE Loss: $0.8011
Epoch 15/100, Test MSE Loss: $0.5090
Epoch 16/100, Test MSE Loss: $0.4636
Epoch 17/100, Test MSE Loss: $0.4340
Epoch 18/100, Test MSE Loss: $0.4741
Epoch 19/100, Test MSE Loss: $0.3998
Epoch 20/100, Test MSE Loss: $0.3781
Epoch 21/100, Test MSE Loss: $0.3525
Epoch 22/100, Test MSE Loss: $0.3754
Epoch 23/100, Test MSE Loss: $0.3124
Epoch 24/100, Test MSE Loss: $0.3174
Epoch 25/100, Test MSE Loss: $0.3034
Epoch 26/100, Test MSE Loss: $0.2722
Epoch 27/100, Te

# Testing

In [99]:
import pandas as pd
model.eval()

test_cases = [
    {
        "name": "At-the-Money (Base)",
        "params": [100.0, 100.0, 1.0, 0.05, 0.2]
    },
    {
        "name": "In-the-Money (ITM)",
        "params": [120.0, 100.0, 1.0, 0.05, 0.2]
    },
    {
        "name": "Out-of-the-Money (OTM)",
        "params": [80.0, 100.0, 1.0, 0.05, 0.2]
    },
    {
        "name": "High Volatility",
        "params": [100.0, 100.0, 1.0, 0.05, 0.5]
    },
    {
        "name": "Short Expiry (1 Month)",
        "params": [100.0, 100.0, 21/252, 0.05, 0.2]
    },
]

results = []

for case in test_cases:
    s, k, t, r, sigma = case["params"]

    true_price = black_scholes(s=s, k=k, t=t, r=r, sigma=sigma)

    test_params_raw = np.array([[s, k, t, r, sigma]])
    test_params_scaled = scaler.transform(test_params_raw)
    input_tensor = torch.tensor(test_params_scaled, dtype=torch.float32).to(device)

    start_time = time.perf_counter()
    with torch.no_grad():
        predicted_price_tensor = model(input_tensor)
    end_time = time.perf_counter()

    predicted_price = predicted_price_tensor.cpu().item()
    inference_time_ms = (end_time - start_time) * 1000

    abs_error = abs(true_price - predicted_price)
    pct_error = (abs_error / true_price) * 100 if true_price > 0 else 0

    results.append({
        "Scenario": case["name"],
        "True Price": f"${true_price:.4f}",
        "Predicted Price": f"${predicted_price:.4f}",
        "Abs Error": f"${abs_error:.4f}",
        "Pct Error": f"{pct_error:.2f}%",
        "Inference Time (ms)": f"{inference_time_ms:.4f}"
    })

results_df = pd.DataFrame(results)
print(results_df.to_string())

                 Scenario True Price Predicted Price Abs Error Pct Error Inference Time (ms)
0     At-the-Money (Base)   $10.4506        $10.0644   $0.3862     3.70%              0.6236
1      In-the-Money (ITM)  $261.6904       $263.2350   $1.5445     0.59%              0.5555
2  Out-of-the-Money (OTM)    $1.8594         $1.7685   $0.0909     4.89%              0.4130
3         High Volatility   $21.7926        $21.9679   $0.1753     0.80%              0.4126
4  Short Expiry (1 Month)    $2.5121         $1.9988   $0.5132    20.43%              0.4263


In [96]:
MODEL_PATH = "option_pricer_model.pth"
torch.save(model.state_dict(), MODEL_PATH)